# Problem Set 4

## Question 2

In this exercise, we numerically study the behaviors of the wild and the pairs bootstrap by considering the following model:
\begin{align}\label{Q4}\tag{1}
Y_i=Z_{1i} + Z_{2i} + Z_{3i} + \varrho_0 Z_{1i} Z_{2i} + (1+\lambda_0 Z_{1i})\epsilon_i~,
\end{align}
for $i=1,\ldots, n$. The sample $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$ is generated as follows.

(1) Generate $V_i,Z_{2i}$ and $Z_{3i}$ from the standard normal distribution with equal correlation $0.2$, and then set
    \begin{align}
    Z_{1i} = \frac{\exp(V_i)-E[\exp(V_i)]}{\sqrt{\mathrm{Var}(\exp(V_i))}}~,
    \end{align}
    where $E[\exp(V_i)]=\exp(0.5)$ and $\sqrt{\mathrm{Var}(\exp(V_i))}=2.16$ (approximately).

(2) Independently, $\epsilon_i$ is generated from a mixture of a $N(-1/9,1)$ variable with probability $0.9$ and a $N(1,4)$ variable with probability $0.1$. When coding, you may first draw a sample $\eta_i$ from a Bernoulli distribution with success probability $0.9$, and then you draw $\epsilon_i$ from $N(-1/9,1)$ if $\eta_i=1$ and otherwise draw $\epsilon_i$ from $N(1,4)$. There may be a Python package for implementing this in a compact fashion, but, if not, I would just set $\epsilon_i=\eta_i W_{1i} + (1-\eta_i)W_{2i}$ for $W_{1i}\sim N(-1/9,1)$ and independently $W_{2i}\sim N(1,4)$.

(3) Generate $Y_i$ according to (1) for each pair $(\varrho_0,\lambda_0)$ to be specified.

Given $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$, we then consider the following regression:
  \begin{align}
  Y_i=\alpha_0+\beta_1Z_{1i} + \beta_2 Z_{2i} + \beta_3 Z_{3i} + U_i~.
  \end{align}
Note that if $\varrho_0=0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]=0$ (the model is correctly specified), and if $\varrho_0\neq 0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]\neq 0$ so the model is misspecified (but one still has $E[U_i(1,Z_{1i},Z_{2i},Z_{3i})]=0$). The model exhibits conditional heteroskedasticity if $\lambda_0\neq 0$. In what follows, set $\lambda_0=1$ and $n=200$.

 You should manually generate the bootstrap samples and compute the bootstrap statistics, as described on p.51 of the lecture notes.

(a) Let $\varrho_0=0$. Compute the empirical rejection rates of the two-sided $t$-test for $\mathrm H_0: \beta_1=1$ at the significance level $\alpha=5\%$, based on the standard normal approximation, the pairs bootstrap and the wild bootstrap using Radamacher weights. Use $1000$ Monte Carlo replications and $200$ bootstrap repetitions. The closer the empirical rejection rates are to $\alpha$ (the nominal rate), the better the test performs. See the pdf of the problem set for a sketch of the coding structure.

In [3]:
# 1) Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as mt
import statsmodels.api as sm
import pandas as pd
from sympy.stats import Rademacher, sample

# 2) Set parameters for the data generating process
n = 200

# The desired mean values of v_i, z2_i and z3_i
mu = np.array([0, 0, 0])

# The desired covariance matrix for v_i, z2_i and z3_i
cov = np.array([
        [ 1,    0.2, 0.2],
        [ 0.2,  1,   0.2],
        [ 0.2,  0.2,   1]
    ])

In [8]:
# 3) Define function for Monte Carlo simulation
# r= number of MC replications, b= number of bootstrap repetitions, n= sample size, beta_1= parameter in null hypothesis 

def mysimul(r, b, n, rho_0, lambda_0, beta_1):
    
    # Preallocate zero vectors for rejection regions
    RejN= np.zeros((r,1))
    RejP= np.zeros((r,1))
    RejW= np.zeros((r,1))
    
    for i in range(r):
        
        # 1) Generate general sample
        rng = np.random.default_rng()
        rv = rng.multivariate_normal(mu, cov, size=n)
        v_i= rv[:,0]
        z2_i= rv[:,1]
        z3_i= rv[:,2]
        # Generate z1_i according to the formula
        z1_i= (np.exp(v_i) - mt.exp(0.5))/2.16
        w1_i = np.random.normal(-1/9, 1, n)
        w2_i = np.random.normal(1, 2, n) #mu, sigma, size
        eta_i = np.random.binomial(size=n, n=1, p= 0.9)
        # Generate e_i according to the formula
        e_i= eta_i*w1_i + (1-eta_i)*w2_i
        # Generate y1_i according to the formula
        y_i= z1_i + z2_i + z3_i + rho_0*z1_i*z2_i + (1+lambda_0*z1_i)*e_i
        # Collect regressors into a dataframe X
        X = pd.DataFrame({'z1': z1_i, 'z2': z2_i, 'z3': z3_i })
        # Add constant
        X = sm.add_constant(X)
        
        # 2) Run regression on general sample
        reg = sm.OLS(y_i, X).fit()
        # Retrieve beta_1_hat from regression
        beta_1_hat= reg.params[1]
        # Retrieve standard error for beta_1_hat from regression
        se= reg.HC0_se[1]
        
        # 3) Compute t-statistic for the general sample
        tn= np.absolute((beta_1_hat - beta_1)/se)
        
        # 4) Generate  Bootstrap samples t-statistics
        ## a) Pairs bootstrap
        BootP0 = []
        for _ in range(b):
            # Create Pd dataframe with general sample for both y and Xs
            mysample = pd.DataFrame({'y_i':y_i,'z1': z1_i, 'z2': z2_i, 'z3': z3_i })
            # Create Pair Bootstrap sample
            mysample_star= mysample.sample(n=200, frac=None, replace=True, weights=None, random_state=None, axis=None, ignore_index=False)
            y_star = mysample_star['y_i']
            X_star = mysample_star[['z1', 'z2', 'z3']]
            # Add constant to X for the regression
            X_star = sm.add_constant(X_star)
            # Run regression
            reg_BP = sm.OLS(y_star, X_star).fit()
            # Retrieve beta_1_hat_BP from regression
            beta_1_hat_BP= reg_BP.params[1]
            # Retrieve standard error for beta_1_hat_BP from regression
            se_BP= reg_BP.HC0_se[1]
            # a.2) Compute t-statistic for Bootstrap-Pairs
            tn_BP= (beta_1_hat_BP - beta_1_hat)/se_BP
            # Store tn_BP values into BootP0
            BootP0.append(tn_BP)
        BootP= np.transpose(np.absolute(np.array(BootP0)))
        
        ## b) Wild bootstrap
        BootW0 = []
        ## For Wild bootstrap, I employ the CLS estimator beta_tilde as suggested in the notes (thanks to Wonjun Choi for the tip!)
        y_new = y_i - beta_1*z1_i
        X_new= pd.DataFrame({'z2': z2_i, 'z3': z3_i })
        # Add constant
        X_new=sm.add_constant(X_new)
        # Run regression to obtain beta_tilde
        reg_CLS=sm.OLS(y_new, X_new).fit()
        beta_tilde= np.array(reg_CLS.params)
        # Generate u_tilde according to the formula u_tilde= y_i - X*beta_tilde
        u_tilde= y_i- np.matmul(X_new,beta_tilde)
        # Generate Wild bootstrap sample
        for _ in range(b):
            # Generate w_i following Rademacher distribution
            rad= Rademacher('W')
            w_i = sample(rad, size= 200)
            # Generate u_tilde_star according to the formula u_tilde_star_i= u_tilde_i*w_i
            u_tilde_star = u_tilde*w_i
            # Generate y_star2 according to the formula y_star2= X*beta_tilde + u_tilde_star
            y_star2= np.matmul(X_new,beta_tilde) +z1_i + u_tilde_star
            # Run regression
            reg_BW = sm.OLS(y_star2, X).fit()
            # Retrieve beta_1_hat_BW from regression
            beta_1_hat_BW= reg_BW.params[1]
            # Retrieve standard error for beta_1_hat_BW from regression
            se_BW= reg_BW.HC0_se[1]
            # Compute t-statistic for Bootstrap-Wild
            tn_BW= (beta_1_hat_BW - beta_1)/se_BW
            # Store tn_BW values into BootW0
            BootW0.append(tn_BW)
        BootW= np.transpose(np.absolute(np.array(BootW0)))
        
        # 5) Generate Bootstrap critical values
        CriticalP= np.quantile(BootP,0.95)
        CriticalW= np.quantile(BootW,0.95)
        
        # 6) Run t-test and store results into RejN, RejP, RejW
        RejN[i] = 1*(tn > 1.96)
        RejP[i] = 1*(tn > CriticalP)
        RejW[i] = 1*(tn > CriticalW)
    
    # 7) Compute average rejection rate
    RejN_mean= np.mean(RejN)
    RejP_mean= np.mean(RejP)
    RejW_mean= np.mean(RejW)
    
    print("The average rejection rate based on the standard normal approximation is", RejN_mean)
    print("The average rejection rate based on the Pair bootstrap is", RejP_mean)
    print("The average rejection rate based on the Wild bootstrap is", RejW_mean)


In [11]:
# Run Montecarlo simulation with 1000 repetitions to test H0: beta_1=1
mysimul(r=1000, b=200, n=200, rho_0=0, lambda_0=1, beta_1=1)

The average rejection rate based on the standard normal approximation is 0.171
The average rejection rate based on the Pair bootstrap is 0.069
The average rejection rate based on the Wild bootstrap is 0.052


(b) Repeat part (a) by setting $\varrho_0\in\{-0.5,0.5\}$ (so the model is misspecified), but now consider the two-sided $t$ test for $\mathrm H_0: \beta_1=1+\varrho_0 b$ for $b=0.41$ (note that $1+\varrho_0 b$ is the slope parameter for $Z_{1i}$ in the best linear prediction of $Y_i$ given $1,Z_{1i},Z_{2i}$, and $Z_{3i}$).

In [9]:
# Set parameters for the misspecified model
rho_0_mis= 0.5
beta_1_mis= 1 + rho_0_mis*0.41
# Run Montecarlo simulation
mysimul(r=1000, b=200, n=200, rho_0=rho_0_mis, lambda_0=1, beta_1=beta_1_mis)

The average rejection rate based on the standard normal approximation is 0.182
The average rejection rate based on the Pair bootstrap is 0.079
The average rejection rate based on the Wild bootstrap is 0.037


In [10]:
# Repeat with rho_0_mis= -0.5
rho_0_mis= -0.5
beta_1_mis= 1 + rho_0_mis*0.41
# Run Montecarlo simulation
mysimul(r=1000, b=200, n=200, rho_0=rho_0_mis, lambda_0=1, beta_1=beta_1_mis)

The average rejection rate based on the standard normal approximation is 0.179
The average rejection rate based on the Pair bootstrap is 0.069
The average rejection rate based on the Wild bootstrap is 0.026
